# StoryLand AI - Modular Demo

**Transform books into travel adventures using modular AI agents.**

This notebook demonstrates the modular version of StoryLand AI, where agents, tools, and services are organized into reusable packages.

## Setup

In [1]:
# Import libraries
import os
import uuid
import logging
import json
from dotenv import load_dotenv

# Import Google ADK components
from google.genai import types
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner

# Import from our modules
from common.config import load_config
from common.logging import setup_logger

from services.session_service import create_session_service
from services.memory_service import create_memory_service
from services.context_manager import ContextManager

from tools.google_books import google_books_tool
from agents.orchestrator import create_workflow

# Load environment
load_dotenv()

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s'
)

logger = setup_logger("storyland_demo")

print("✅ Environment Setup Complete")
print(f"   Google API Key: {'OK' if os.getenv('GOOGLE_API_KEY') else 'MISSING'}")

✅ Environment Setup Complete
   Google API Key: OK


## Configuration

Configure your book and session settings here:

In [2]:
# ============================================================
# CONFIGURE YOUR BOOK HERE
# ============================================================
BOOK_TITLE = "gone with the wind"
AUTHOR = ""  # Optional - leave empty if unknown

# Session configuration
USE_DATABASE = False  # Set to True to use SQLite for persistent sessions
USE_MEMORY = False    # Set to True to enable memory-based personalization
USER_ID = "user1"
# ============================================================

print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}")
print(f"Database: {'Enabled' if USE_DATABASE else 'Disabled'}")
print(f"Memory: {'Enabled' if USE_MEMORY else 'Disabled'}")

Book: gone with the wind
Author: Unknown
Database: Disabled
Memory: Disabled


## Initialize Services

Create the model, session service, and workflow:

In [3]:
# Load configuration
config = load_config()

# Configure Gemini model
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

model = Gemini(
    model=config.model_name,
    api_key=config.google_api_key,
    retry_options=retry_config
)

print(f"✅ Model configured: {model.model}")

# Create session service
session_service = create_session_service(use_database=USE_DATABASE)
print("✅ Session service created")

# Create memory service (if enabled)
memory_service = None
if USE_MEMORY:
    memory_service = create_memory_service()
    print("✅ Memory service created")

# Create context manager
context_manager = ContextManager(max_events=20)
print("✅ Context manager created")

# Create workflow
workflow = create_workflow(model, google_books_tool)
print("✅ Workflow created")

# Create runner
runner = Runner(
    agent=workflow,
    app_name="storyland",
    session_service=session_service
)
print("✅ Runner created")

2025-11-20 22:32:57,448 - WARNING - google_adk.google.adk.runners - App name mismatch detected. The runner is configured with app name "storyland", but the root agent was loaded from "/Users/osa/Documents/storyland/storyland-ai/.venv/lib/python3.12/site-packages/google/adk/agents", which implies app name "agents".


✅ Model configured: gemini-2.0-flash-lite
Using InMemorySessionService (not persistent)
✅ Session service created
✅ Context manager created
✅ Workflow created
✅ Runner created


## Create Session

In [4]:
# Create a new session
session_id = str(uuid.uuid4())

await session_service.create_session(
    app_name="storyland",
    user_id=USER_ID,
    session_id=session_id,
    state={
        "book_title": BOOK_TITLE,
        "author": AUTHOR,
        # You can add user preferences here
        "user:preferences": {
            "prefers_museums": True,
            "budget": "moderate",
            "pace": "moderate"
        }
    }
)

print(f"✅ Session created: {session_id[:8]}...")

✅ Session created: ea03edf7...


## Execute Workflow

Run the multi-agent workflow to create the travel itinerary:

In [5]:
print("="*70)
print("EXECUTING WORKFLOW")
print("="*70)
print(f"Book: {BOOK_TITLE}")
print(f"Author: {AUTHOR or 'Unknown'}\n")

# Create the prompt
prompt = f"""Create a literary travel itinerary for "{BOOK_TITLE}" by {AUTHOR or 'unknown author'}.

Execute all steps and return the complete combined results."""

user_message = types.Content(
    role='user',
    parts=[types.Part(text=prompt)]
)

print(f"USER REQUEST:")
print(f"{prompt}\n")
print("Starting workflow execution...\n")
print("="*70)

# Run workflow and collect results
final_response = None
event_count = 0

async for event in runner.run_async(
    user_id=USER_ID,
    session_id=session_id,
    new_message=user_message
):
    event_count += 1
    
    # Show progress
    if event.author:
        print(f"[{event_count}] {event.author}")
    
    if event.is_final_response():
        final_response = event
        print(f"\n✅ Workflow complete ({event_count} events)")

print("\n" + "="*70)

2025-11-20 22:33:28,987 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False


EXECUTING WORKFLOW
Book: gone with the wind
Author: Unknown

USER REQUEST:
Create a literary travel itinerary for "gone with the wind" by unknown author.

Execute all steps and return the complete combined results.

Starting workflow execution...



2025-11-20 22:33:29,669 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 22:33:29,673 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-20 22:33:29,674 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-20 22:33:29,675 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


[1] book_metadata_researcher

✅ Workflow complete (1 events)


2025-11-20 22:33:30,377 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 22:33:30,383 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-20 22:33:30,389 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-20 22:33:30,390 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


...

[9] city_formatter

✅ Workflow complete (9 events)


2025-11-20 22:33:57,583 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 22:33:57,586 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.
2025-11-20 22:33:57,590 - INFO - google_adk.google.adk.models.google_llm - Sending out request, model: gemini-2.0-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
2025-11-20 22:33:57,590 - INFO - google_genai.models - AFC is enabled with max remote calls: 10.


[10] landmark_formatter

✅ Workflow complete (10 events)


2025-11-20 22:34:03,731 - INFO - httpx - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash-lite:generateContent "HTTP/1.1 200 OK"
2025-11-20 22:34:03,738 - INFO - google_adk.google.adk.models.google_llm - Response received from the model.


[11] trip_composer

✅ Workflow complete (11 events)



## Extract and Display Results

In [6]:
# Extract result from final response
result_data = {}
if final_response and final_response.content and final_response.content.parts:
    for part in final_response.content.parts:
        if hasattr(part, 'text') and part.text:
            json_start = part.text.find('{')
            json_end = part.text.rfind('}') + 1
            
            if json_start >= 0 and json_end > json_start:
                try:
                    result_data = json.loads(part.text[json_start:json_end])
                    break
                except json.JSONDecodeError as e:
                    logger.error(f"Failed to parse JSON: {e}")

# Display the itinerary
cities_list = result_data.get('cities', [])

if cities_list:
    print(f"\nTRAVEL ITINERARY - {len(cities_list)} Cities")
    print("="*70)
    
    for i, city_plan in enumerate(cities_list, 1):
        print(f"\nCity #{i}: {city_plan.get('name')}, {city_plan.get('country')}")
        print(f"Days suggested: {city_plan.get('days_suggested', 1)}")
        
        if city_plan.get('overview'):
            print(f"\nOverview: {city_plan.get('overview')}")
        
        stops = city_plan.get('stops', [])
        if stops:
            print(f"\nStops ({len(stops)}):")
            for j, stop in enumerate(stops, 1):
                print(f"  {j}. {stop.get('name')} ({stop.get('type')})")
                print(f"     {stop.get('reason')}")
    
    # Summary
    summary_text = result_data.get('summary_text')
    if summary_text:
        print("\n" + "="*70)
        print("TRIP SUMMARY")
        print("="*70)
        print(f"\n{summary_text}\n")
else:
    print("\n❌ No itinerary data found")


TRAVEL ITINERARY - 3 Cities

City #1: Atlanta, USA
Days suggested: 3

Overview: Immerse yourself in the world of "Gone with the Wind" in Atlanta, the heart of the story. Explore the city's historical sites, tracing the steps of Scarlett O'Hara and witnessing the transformation of the South during the Civil War and Reconstruction eras. Discover the places that inspired the novel and film, and experience the era's atmosphere.

Stops (5):
  1. Margaret Mitchell House (museum)
     This is where Margaret Mitchell wrote "Gone with the Wind". Step into the rooms where the story came to life, and gain insight into the author's creative process.
  2. Atlanta History Center (museum)
     Explore Civil War exhibits and view the Margaret Mitchell House, offering a comprehensive view of the era and the author's legacy.
  3. Historic Oakland Cemetery (cemetery)
     Visit the final resting place of Margaret Mitchell. Reflect on the story while strolling through the historic grounds.
  4. Georgian 

## Save to Memory (Optional)

If memory is enabled, save this session for future personalization:

In [7]:
if memory_service:
    # Get the full session
    session = await session_service.get_session(
        app_name="storyland",
        user_id=USER_ID,
        session_id=session_id
    )
    
    # Add to memory
    await memory_service.add_session_to_memory(session)
    print("✅ Session added to memory for future personalization")
    
    # Test memory search
    memory_results = await memory_service.search_memory(
        app_name="storyland",
        user_id=USER_ID,
        query="travel preferences"
    )
    
    print(f"\nMemory search found {len(memory_results.memories)} relevant memories")
else:
    print("ℹ️  Memory service not enabled")

ℹ️  Memory service not enabled


## Summary

This notebook demonstrates the modular architecture:

- **Models** ([models/](models/)) - Pydantic data structures
- **Tools** ([tools/](tools/)) - External API integrations
- **Agents** ([agents/](agents/)) - Specialized AI agents
- **Services** ([services/](services/)) - Session, memory, and context management
- **Config** ([common/config.py](common/config.py)) - Centralized configuration

### Next Steps

1. **Try the CLI**: `python main.py "your book title" --author "Author Name"`
2. **Enable persistence**: Set `USE_DATABASE=true` in `.env`
3. **Enable memory**: Set `USE_MEMORY=true` in `.env`
4. **Customize agents**: Edit files in [agents/](agents/) directory
5. **Add new tools**: Create new tools in [tools/](tools/) directory